In [12]:
import pickle
import joblib
from keras.models import load_model
import pandas as pd
import numpy as np

In [13]:
remain_features = ['CG',
                'AT',
                'peak_Fourier',
                'none_levated_peak_Fourier',
                'k1',
                'GC',
                'fickett_score-ORF',
                'ACG',
                'CGT',
                'TA',
                'TAT',
                'variance_Accumulated',
                'GGG',
                'k2',
                'CTT',
                'maximum_Accumulated',
                'percentile75_Fourier',
                'CGC',
                'AC',
                'k3',
                'ATA',
                'fickett_score-full-sequence',
                'TTC',
                'GG',
                'TTT',
                'CC',
                'percentile50_Fourier',
                'k4',
                'TC',
                'semi_interquartile_range_Fourier',
                'k5',
                'TT',
                'variance_Fourier',
                'average_GC_content_ORF',
                'seq_overlap',
                'seq_med',
                'n_start',
                'n_end',
                'label']

In [21]:
model = load_model('keras_model.h5')

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

scaler = joblib.load('standard_scaler.pkl')
start_scaler = joblib.load('start_min_max_scaler.pkl')
end_scaler = joblib.load('end_min_max_scaler.pkl')

pca = joblib.load('pca.pkl')

In [24]:
df = pd.read_csv(open("df_test.csv", 'rb'))
df = df[remain_features + ['seq_overlap', 'seq_med', 'n_start', 'n_end', 'bais']]

In [25]:
df.head()

,CG,AT,peak_Fourier,none_levated_peak_Fourier,k1,GC,fickett_score-ORF,ACG,CGT,TA,...,seq_overlap,seq_med,n_start,n_end,label,seq_overlap,seq_med,n_start,n_end,bais
0,0.076923,0.050167,0.286533,6.192460,1.994425,0.086957,1.2324,0.013423,0.013423,0.026756,...,AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATT...,CTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGT...,0,299,1,AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATT...,CTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGT...,0,299,300
1,0.066890,0.070234,0.286533,6.144891,1.992519,0.100334,1.2404,0.010067,0.010067,0.026756,...,TAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGG...,ATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCT...,0,299,1,TAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGG...,ATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCT...,0,299,400
2,0.100334,0.073579,0.280112,6.243568,1.996839,0.103679,1.1872,0.013423,0.016779,0.040134,...,TAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGT...,TAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATT...,0,299,1,TAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGT...,TAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATT...,0,299,500
3,0.086957,0.086957,0.275482,6.046975,1.998023,0.096990,1.1674,0.006711,0.020134,0.040134,...,CTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGT...,CAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATG...,0,299,1,CTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGT...,CAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATG...,0,299,600
4,0.103679,0.063545,0.293255,6.210104,1.985911,0.103679,1.2395,0.013423,0.016779,0.040134,...,ATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCT...,TGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTATGGCCGGCGT...,0,299,1,ATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCT...,TGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTATGGCCGGCGT...,0,299,700


In [41]:
X_seq1 = df['seq_overlap'].iloc[:, 0]
X_seq2 = df['seq_med'].iloc[:, 1]
X = df.drop(['seq_overlap', 'seq_med', 'n_start', 'n_end', 'bais', 'label'], axis = 1)

X = scaler.transform(X)
X = pca.transform(X)

X_seq1 = np.array(tokenizer.texts_to_sequences(X_seq1))
X_seq2 = np.array(tokenizer.texts_to_sequences(X_seq2))

y_train_start = start_scaler.transform(df['n_start'].values.reshape(-1, 1))
y_train_end = end_scaler.transform(df['n_end'].values.reshape(-1, 1))

In [42]:
predictions = model.predict([X_seq1, X_seq2, X])
predictions[1] = np.minimum(predictions[1], 1)
predictions[1] = np.maximum(predictions[1], 0)
predictions[1] = start_scaler.inverse_transform(predictions[1])

predictions[2] = np.minimum(predictions[2], 1)
predictions[2] = np.maximum(predictions[2], 0)
predictions[2] = end_scaler.inverse_transform(predictions[2])

proba = predictions[0]
df['predicted_lable'] = np.argmax(predictions[0], axis = 1)
df['predictions_proba'] = proba[:, 1]
df['predicted_start'] = predictions[1]
df['predicted_end'] = predictions[2]

1451/1451 [==============================] - 27s 13ms/step


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46405 entries, 0 to 46404
Data columns (total 48 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CG                                46405 non-null  float64
 1   AT                                46405 non-null  float64
 2   peak_Fourier                      46405 non-null  float64
 3   none_levated_peak_Fourier         46405 non-null  float64
 4   k1                                46405 non-null  float64
 5   GC                                46405 non-null  float64
 6   fickett_score-ORF                 46405 non-null  float64
 7   ACG                               46405 non-null  float64
 8   CGT                               46405 non-null  float64
 9   TA                                46405 non-null  float64
 10  TAT                               46405 non-null  float64
 11  variance_Accumulated              46405 non-null  float64
 12  GGG 

In [44]:
df.to_csv("final.csv")

In [55]:
annotation = "# PGAM\n# PGAM version 1.0\n"

In [58]:
n_window = 300
id_transcript = 1

local_start = 0
local_end = 0
prev_label = 0
global_proba = 0
with open("annotations.gtf", "w") as gtf_file:
    gtf_file.write(annotation)
    for start, lable, proba in zip(df['bais'], df['predicted_lable'], df['predictions_proba']):
        if lable == 1 and local_start == 0:
            local_start = start
            global_proba = proba
        if lable == 0 and prev_label == 1:
            local_end = start + n_window

        if local_start != 0 and local_end != 0:
            annotation_line = f"U00096.3\tPGAM\ttranscript\t{local_start}\t{local_end}\t{global_proba * 100:.5f}\t.\t.\tgene_id \"MSTRG.{id_transcript}\";\ttranscript_id \"MSTRG.{id_transcript}\";\n"
            gtf_file.write(annotation_line)
            annotation_line = f"U00096.3\tPGAM\texon\t{local_start}\t{local_end}\t{global_proba * 100}\t.\t.\tgene_id \"MSTRG.{id_transcript}\";\ttranscript_id \"MSTRG.{id_transcript}\";\n"
            gtf_file.write(annotation_line)
            id_transcript += 1

            local_start = 0
            local_end = 0

        global_proba = np.mean([global_proba, proba])
        prev_label = lable
    

# PGAM
# PGAM version 1.0
U00096.3	PGAM	transcript	300	5900	84.27938	.	.	gene_id "MSTRG.1";	transcript_id "MSTRG.1";
U00096.3	PGAM	exon	300	5900	84.27938222885132	.	.	gene_id "MSTRG.1";	transcript_id "MSTRG.1";
U00096.3	PGAM	transcript	5700	6200	78.10004	.	.	gene_id "MSTRG.2";	transcript_id "MSTRG.2";
U00096.3	PGAM	exon	5700	6200	78.10004353523254	.	.	gene_id "MSTRG.2";	transcript_id "MSTRG.2";
U00096.3	PGAM	transcript	6000	6600	87.92967	.	.	gene_id "MSTRG.3";	transcript_id "MSTRG.3";
U00096.3	PGAM	exon	6000	6600	87.92966604232788	.	.	gene_id "MSTRG.3";	transcript_id "MSTRG.3";
U00096.3	PGAM	transcript	6500	11600	80.13731	.	.	gene_id "MSTRG.4";	transcript_id "MSTRG.4";
U00096.3	PGAM	exon	6500	11600	80.13731390237753	.	.	gene_id "MSTRG.4";	transcript_id "MSTRG.4";
U00096.3	PGAM	transcript	11500	17400	84.85782	.	.	gene_id "MSTRG.5";	transcript_id "MSTRG.5";
U00096.3	PGAM	exon	11500	17400	84.8578244447708	.	.	gene_id "MSTRG.5";	transcript_id "MSTRG.5";
U00096.3	PGAM	transcript	17400	18900